In [1]:
run fixpath.py

In [2]:
# from src.generator import data_generator, clip_generator, label_generator
from src.data import player_data
from src.util import characters, id_from_char, char_from_id
from src.export import display_progress

from pymongo import MongoClient
from tensorflow import keras

import numpy as np
import pickle

In [3]:
for character in characters:
    print(character)

CAPTAIN_FALCON
DONKEY_KONG
FOX
GAME_AND_WATCH
KIRBY
BOWSER
LINK
LUIGI
MARIO
MARTH
MEWTWO
NESS
PEACH
PIKACHU
ICE_CLIMBERS
JIGGLYPUFF
SAMUS
YOSHI
ZELDA
SHEIK
FALCO
YOUNG_LINK
DR_MARIO
ROY
PICHU
GANONDORF


# Pretrained Model

Base models are trained in cnn.ipynb

In [4]:
# to load the saved model
model = keras.models.load_model('../models/custom_mk4')
model.summary()

Model: "custom_mk4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv1D)               (None, None, 150)         58650     
_________________________________________________________________
batch_normalization (BatchNo (None, None, 150)         600       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 150)         0         
_________________________________________________________________
conv2 (Conv1D)               (None, None, 256)         576256    
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 256)         1024      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 256)         0         
_________________________________________________________________
conv3 (Conv1D)               (None, None, 512)         1

# Repurposing

In [5]:
len(model.layers)

22

In [6]:
for _ in range(9):
    model.pop()

for layer in model.layers:
    layer.trainable = False
    
model.summary()

Model: "custom_mk4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv1D)               (None, None, 150)         58650     
_________________________________________________________________
batch_normalization (BatchNo (None, None, 150)         600       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 150)         0         
_________________________________________________________________
conv2 (Conv1D)               (None, None, 256)         576256    
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 256)         1024      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 256)         0         
_________________________________________________________________
conv3 (Conv1D)               (None, None, 512)         1

In [7]:
from tensorflow.keras.activations import swish
from tensorflow.keras.layers import Dense, BatchNormalization, Activation, Dropout
from tensorflow.keras.optimizers import Adam, Nadam
# adam = Adam(learning_rate=.0001)
# nadam = Nadam(learning_rate=.0001)

model.add(Dense(64, use_bias=True, name='Blynde_dense1'))
model.add(BatchNormalization(name='Blynde_BatchNorm1'))
model.add(Activation(swish, name='Blynde_Activation1'))
model.add(Dropout(.5, name='Blynde_Dropout1'))

model.add(Dense(64, use_bias=True, name='Blynde_dense2'))
model.add(BatchNormalization(name='Blynde_BatchNorm2'))
model.add(Activation(swish, name='Blynde_Activation2'))
model.add(Dropout(.5, name='Blynde_Dropout2'))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])

In [8]:
model.summary()

Model: "custom_mk4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv1D)               (None, None, 150)         58650     
_________________________________________________________________
batch_normalization (BatchNo (None, None, 150)         600       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 150)         0         
_________________________________________________________________
conv2 (Conv1D)               (None, None, 256)         576256    
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 256)         1024      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 256)         0         
_________________________________________________________________
conv3 (Conv1D)               (None, None, 512)         1

In [9]:
# training data
data_train = player_data(player_dir = '../data/player/Blynde/train',
                         nonplayer_dir = '../data/character/train',
                         batch_size = 32)
# testing data
data_test = player_data(player_dir = '../data/player/Blynde/test',
                        nonplayer_dir = '../data/character/test',
                        batch_size = 32)

In [10]:
# Training Loop
model.fit(data_train,
          epochs=10,
          steps_per_epoch=256,
          verbose=1)

score = model.evaluate(data_test, verbose=1, steps=25)
print('\nTest score:', round(score[0], 3))
print(f'Test accuracy: {round(score[1]*100)}%')

Epoch 1/10
256/256 [==============================] - 11s 43ms/step - loss: 0.5417 - binary_accuracy: 0.7256
Epoch 2/10
256/256 [==============================] - 11s 42ms/step - loss: 0.4347 - binary_accuracy: 0.8041
Epoch 3/10
256/256 [==============================] - 11s 43ms/step - loss: 0.3951 - binary_accuracy: 0.8293
Epoch 4/10
256/256 [==============================] - 11s 43ms/step - loss: 0.3739 - binary_accuracy: 0.8387
Epoch 5/10
256/256 [==============================] - 11s 43ms/step - loss: 0.3622 - binary_accuracy: 0.8425
Epoch 6/10
256/256 [==============================] - 11s 43ms/step - loss: 0.3533 - binary_accuracy: 0.8461
Epoch 7/10
256/256 [==============================] - 11s 43ms/step - loss: 0.3439 - binary_accuracy: 0.8503
Epoch 8/10
256/256 [==============================] - 11s 43ms/step - loss: 0.3357 - binary_accuracy: 0.8514
Epoch 9/10
256/256 [==============================] - 11s 43ms/step - loss: 0.3223 - binary_accuracy: 0.8623
Epoch 10/10
25/25 [

In [13]:
# training data
data_train = player_data(player_dir = '../data/player/Blynde/train',
                         nonplayer_dir = '../data/character/train',
                         batch_size = 256)

In [14]:
# Training Loop
model.fit(data_train,
          epochs=10,
          steps_per_epoch=32,
          verbose=1)

score = model.evaluate(data_test, verbose=1, steps=25)
print('\nTest score:', round(score[0], 3))
print(f'Test accuracy: {round(score[1]*100)}%')

Epoch 1/10
32/32 [==============================] - 12s 370ms/step - loss: 0.3079 - binary_accuracy: 0.8721
Epoch 2/10
32/32 [==============================] - 12s 374ms/step - loss: 0.3009 - binary_accuracy: 0.8702
Epoch 3/10
32/32 [==============================] - 12s 364ms/step - loss: 0.2983 - binary_accuracy: 0.8783
Epoch 4/10
32/32 [==============================] - 12s 365ms/step - loss: 0.2988 - binary_accuracy: 0.8727
Epoch 5/10
32/32 [==============================] - 12s 364ms/step - loss: 0.2941 - binary_accuracy: 0.8754
Epoch 6/10
32/32 [==============================] - 12s 362ms/step - loss: 0.2925 - binary_accuracy: 0.8766
Epoch 7/10
32/32 [==============================] - 12s 363ms/step - loss: 0.3023 - binary_accuracy: 0.8765
Epoch 8/10
32/32 [==============================] - 12s 363ms/step - loss: 0.2879 - binary_accuracy: 0.8801
Epoch 9/10
32/32 [==============================] - 12s 367ms/step - loss: 0.2893 - binary_accuracy: 0.8777
Epoch 10/10
25/25 [=========